
# Задание:
----

<br>

Вам предоставлен анонимизированные результаты **реального A/B тестирования**, проходившего когда-то в нашем отделе (`ab_test_ensembles.csv`). В тесте было два варианта работы сервиса - `standard` и `svm_ensemble_v_1_22`, где работали несколько моделей классификации для целей сервиса.

<br>


------
Вам, как специалистам по машинному обучению предстоит ответить на **2 главных вопроса:**

1. Стоит ли нам оставить старый вариант работы сервиса или заменить его на вариант работы с моделями классификации (используем всю выборку 200к+ пользователей).
2. Кроме того, посчитайте вывод для типа пользователей (`user_type`). Стоит ли для новых (старых) пользователей оставить старый (новый) вариант работы сервиса.


<br>

Для того, чтобы освежить в памяти процесс тестирования статистических гипотез, непомню, что тестирование состоит из следующих частей:
1. Дизайн эксперимента.
2. Подготовка и запуск эксперимента.
3. Сбор данных и аналитика полученных данных.
4. Визуализация результатов тестирования.
5. Тестирование гипотез.
6. Вывод и интерпритация результатов.

-----

#### Критерии оценки задания:

1. **Первое, что будет проверяться - вывод полученных результатов**, в случае если выводы сделаны не правильно, задание считается проваленным и на этом этап проверки заканчивается (пропускаются этапы код-ревью, оформления и визуализации, качество кода).
2. В случае если результаты и интерпритация результатов оказались верны, проводится код-ревью и проверка этапа визуализаций, поиск проблемных точек, точек роста.
3. **Максимальный балл** который можно получить, выполнив текущее задание: **2 балла за 1 вопрос, 2 балла за 2 вопрос и 1 балл за эффективный и аккуратный код.


Удачи, примените все свои навыки, которые вам доступны на данный момент и покажите на что вы способны!

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [2]:
df = pd.read_csv('./ab_test_ensembles.csv')

In [26]:
df.head()

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    294478 non-null  object
 1   timestamp  294478 non-null  object
 2   group      294478 non-null  object
 3   variants   294478 non-null  object
 4   converted  294478 non-null  int64 
 5   location   294478 non-null  object
 6   age_group  294478 non-null  object
 7   user_type  294478 non-null  object
dtypes: int64(1), object(7)
memory usage: 18.0+ MB


In [6]:
df.isnull().sum()

user_id      0
timestamp    0
group        0
variants     0
converted    0
location     0
age_group    0
user_type    0
dtype: int64

In [31]:
df.describe(include='all')

,user_id,timestamp,group,variants,converted,location,age_group,user_type
count,294478,294478,294478,294478,294478.000000,294478,294478,294478
unique,294478,294478,2,2,NaN,1,6,2
top,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,treatment,standard,NaN,United Kingdom,26-33,registered_user
freq,1,1,147276,147239,NaN,294478,49270,147240
mean,NaN,NaN,NaN,NaN,0.119659,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,0.324563,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


Датафрейм имеет 294478 значений, нет пропущенных значений, данные пользователей не дублируются, чистка не требуется, можно приступать к ab-тестированию

In [34]:
df.group.value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

In [32]:
df.variants.value_counts()

standard               147239
svm_ensemble_v_1_22    147239
Name: variants, dtype: int64

In [33]:
df.converted.value_counts()

0    259241
1     35237
Name: converted, dtype: int64

In [35]:
df.groupby('group').variants.value_counts()

group      variants           
control    standard               145274
           svm_ensemble_v_1_22      1928
treatment  svm_ensemble_v_1_22    145311
           standard                 1965
Name: variants, dtype: int64

У нас есть две группы: контрольная (147 202 значений) и treatment (147 276 значений), которая была подвергнута изменениям. Каждая из этих групп содержит два варинта пользования сервисом: стандартный (standard) и обновленный (svm_ensemble_v_1_22). При этом контрольная группа содержит 145 274 значения стандартной работы сервиса и 1928 значений для обновленного сервиса, а treatment-группа содержит 145 311 значений обновленного сервиса и 1965 значений стандартного сервиса. Для всех исптытаний есть результат converted, который принимает значение или 0 или 1. Группа А (контрольная группа) использует старый вариант работы сервиса, группа В (экспериментальная) использует новый вариант работы сервиса. Для анализа будем использовать данные контрольной группы со стандартным вариантом работы сервиса и экспериментальной группы с обновленным вариантом работы сервиса. Создадим датафрейм, отсортированный по этим данным.

In [82]:
analyzed_df = df[(df['group'] == 'control') & (df['variants'] == 'standard') |
          (df['group'] == 'treatment') & (df['variants'] == 'svm_ensemble_v_1_22')]
analyzed_df

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user
...,...,...,...,...,...,...,...,...
294473,497bf0f8-0092-4736-8ae7-775b6cc9736c,2022-05-03 22:28:38.630509,control,standard,0,United Kingdom,58+,new_user
294474,6cb20e62-e49d-41cf-97da-0e11cb439dca,2022-05-12 00:51:57.078372,control,standard,0,United Kingdom,42-49,new_user
294475,d307b0ad-92a1-409c-a2d2-da8f4a118576,2022-05-22 11:45:03.439544,control,standard,0,United Kingdom,18-25,registered_user
294476,c6bd8da5-2114-4fd4-92c6-e4b11a8be4dc,2022-05-15 01:20:28.957438,control,standard,0,United Kingdom,50-57,new_user


In [83]:
analyzed_df.groupby('group').variants.value_counts()

group      variants           
control    standard               145274
treatment  svm_ensemble_v_1_22    145311
Name: variants, dtype: int64

Рассчитаем среднее значение показателя converted

In [84]:
mean_converted = analyzed_df.converted.mean()
mean_converted

0.11959667567149027

Найдем среднее значение converted для каждой группы

In [85]:
analyzed_df.groupby('group').aggregate({'converted': 'mean'})

,converted
group,
control,0.120386
treatment,0.118807


Средний показатель converted равен 0,1196.
Средний показатель converted для контрольной группы равен 0,1204.
Средний показатель converted для экспериментальной группы равен 0,1189.
У экспериментально группы показатель converted ниже, чем у контрольной.

Определим, есть ли существенная разница между этими группами. Для расчета p-value будем использовать метод хи-квадрат Пирсона, т.к. у нас дискретные данные и большое количество наблюдений.

Нулевая гипотеза (H0): не существует статистической значимости между двумя выборками  
Альтернативная гипотеза (Н1): существует статистически значимая разница между двумя выборками  
alpha = 0.05 - уровень значимости

Создадим таблицу сопряженности для расчета коэффициентов

In [86]:
cross_table = pd.crosstab(analyzed_df["variants"], analyzed_df["converted"])
cross_table

converted,0,1
variants,,
standard,127785,17489
svm_ensemble_v_1_22,128047,17264


In [87]:
def chi2_con_test(data):
    chi2, p, dof, ex = chi2_contingency(data)
    alpha = 0.05

    print('p=%.4f , alpha=%.2f '%(p,alpha))
    if p > alpha:
        print('Two variants have no significant difference')
    else:
        print('Two variantss have a significant difference')

In [88]:
chi2_con_test(cross_table)

p=0.1916 , alpha=0.05 
Two variants have no significant difference


Вывод: так как p-value больше 0,05, значит, что наши выборки не имеют статистически значимый различий и подтвердилась нулевая гипотеза Н0. У стандартного варината работы сервиса и обновленного варинта работы сервиса существенных различий нет.

Далее определим есть ли существенная разница в старом и новом варианте работы сервиса для старых и новых пользователей

In [70]:
df.user_type.value_counts()

registered_user    147240
new_user           147238
Name: user_type, dtype: int64

In [73]:
old_users_df = df[(df['user_type'] == 'registered_user')]
old_users_df

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user
5,426f77eb-8739-43b6-999c-aecd14d9f346,2022-05-10 15:20:49.083499,control,standard,0,United Kingdom,42-49,registered_user
...,...,...,...,...,...,...,...,...
294464,9c1f18e9-b20f-4ef2-ae8d-3a190c93e5c3,2022-05-17 01:51:56.106436,control,standard,0,United Kingdom,34-41,registered_user
294468,873f6fd0-8f8b-47c9-b38e-b18b65262430,2022-05-02 19:20:05.460595,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
294470,bd11ea90-58e6-4e15-b9b7-1e45bb83475a,2022-05-11 02:42:21.195145,control,standard,0,United Kingdom,34-41,registered_user
294475,d307b0ad-92a1-409c-a2d2-da8f4a118576,2022-05-22 11:45:03.439544,control,standard,0,United Kingdom,18-25,registered_user


In [78]:
old_users_df.variants.value_counts()

svm_ensemble_v_1_22    73798
standard               73442
Name: variants, dtype: int64

Для уже зарегистрированных пользователей примерно одинаковые количества стандартной работы сервиса и обновленной

In [79]:
mean_old_user_converted = old_users_df.converted.mean()
mean_old_user_converted

0.12016435751154578

In [80]:
old_users_df.groupby('variants').aggregate({'converted': 'mean'})

,converted
variants,
standard,0.119877
svm_ensemble_v_1_22,0.120450


Среднее значение converted для зарегистрированных пользователей равно 0.1206, для стандартной работы сервиса - 0.1198, для обновленной работы сервиса - 0.1204.
Определим, есть ли существнная разница между выборками, составив таблицу сопряженности и рассчитав p-value методом хи-квадрат Пирсона

In [89]:
old_users_cross_table = pd.crosstab(old_users_df["variants"], old_users_df["converted"])
old_users_cross_table

converted,0,1
variants,,
standard,64638,8804
svm_ensemble_v_1_22,64909,8889


In [90]:
chi2_con_test(old_users_cross_table)

p=0.7411 , alpha=0.05 
Two variants have no significant difference


Вывод: т.к. p-value значительно превышает 0.05, ожем сделать вывод, что нет существенной разницы стандартной работы сервиса и обновленной для зарегистрированных пользователей. Принимаем нулевую гипотезу о равенстве двух выборок.

Далее определим, есть ли существенная разница в стандартной и обновленной работе сервиса для новых пользователей.

In [92]:
new_users_df = df[(df['user_type'] == 'new_user')]
new_users_df

,user_id,timestamp,group,variants,converted,location,age_group,user_type
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
8,070f398e-e6bf-43e7-88bb-37eff2cb9212,2022-05-04 17:58:08.979471,treatment,svm_ensemble_v_1_22,1,United Kingdom,18-25,new_user
9,9cff0b7e-d42b-4508-9fbe-a4e74f7f29c5,2022-05-15 18:11:06.610965,treatment,svm_ensemble_v_1_22,1,United Kingdom,26-33,new_user
11,1cc7ebbb-efb9-43a6-9230-11161ae910ba,2022-05-21 22:37:47.774891,treatment,svm_ensemble_v_1_22,0,United Kingdom,58+,new_user
14,137d7bf0-309c-4aad-ad9e-302a2f45dc70,2022-05-22 11:45:11.327945,treatment,svm_ensemble_v_1_22,0,United Kingdom,50-57,new_user
...,...,...,...,...,...,...,...,...
294471,31b72c34-b5e6-412d-a651-2d68e40294b7,2022-05-21 22:44:20.378320,control,standard,0,United Kingdom,50-57,new_user
294472,28897e32-65de-4fd0-8b57-66f1dc1cab3f,2022-05-04 03:36:46.071379,treatment,svm_ensemble_v_1_22,0,United Kingdom,18-25,new_user
294473,497bf0f8-0092-4736-8ae7-775b6cc9736c,2022-05-03 22:28:38.630509,control,standard,0,United Kingdom,58+,new_user
294474,6cb20e62-e49d-41cf-97da-0e11cb439dca,2022-05-12 00:51:57.078372,control,standard,0,United Kingdom,42-49,new_user


In [94]:
new_users_df.variants.value_counts()

standard               73797
svm_ensemble_v_1_22    73441
Name: variants, dtype: int64

In [95]:
mean_new_user_converted = new_users_df.converted.mean()
mean_new_user_converted

0.11915402273869517

In [96]:
new_users_df.groupby('variants').aggregate({'converted': 'mean'})

,converted
variants,
standard,0.121075
svm_ensemble_v_1_22,0.117223


Среднее значение onverted для новых пользователей равно 0.1191, со стандартной работой сервиса - 0.1210, с обновленной работой сервиса - 0.1172. Аналогично определим, есть ли существенная разница между выборками.

In [97]:
new_users_cross_table = pd.crosstab(new_users_df["variants"], new_users_df["converted"])
new_users_cross_table

converted,0,1
variants,,
standard,64862,8935
svm_ensemble_v_1_22,64832,8609


In [98]:
chi2_con_test(new_users_cross_table)

p=0.0230 , alpha=0.05 
Two variantss have a significant difference


**Вывод:** так как p-value меньше 0.05 мы отвергаем нулевую гипотезу и принимает альтернативную гипотезу о том, что две выборки не равны и существуем значимые различия между ними. Среднее значение converted выше у стандартной работы сервиса, следовательно, этот вариант работы выгоднее.  
Учитывая, что нет существенной разницы в стандартной и обновленной работе сервиса для контрольной и экспериментальной группы, а также для зарегистрированных пользователей, но стандартный вариант работы сервиса лучше для новых пользователей (больше показатель converted, следовательно, больше покупок), то нужно оставить стандартных вариант работы сервиса.